In [1]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [48]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_3a3d1a680f0f4624a339932d749da171.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3,4,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
1,2,3,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
2,3,3,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,3,3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
4,2,3,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1


In [49]:
df.shape

(148, 19)

In [50]:
df=df.rename(columns={0:"class"})
df.head()

,class,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,3,4,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
1,2,3,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
2,3,3,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,3,3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
4,2,3,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1


In [51]:
df.groupby("class")["class"].count()

class
1     2
2    81
3    61
4     4
Name: class, dtype: int64

For now, I am going to drop Class 1, which has only 2 instances, and Class 4, which has only 4 instances. They will be included during the reinforcement round of training if the mdoel han handle them. If not, they will be counted as missclassified when computing accracy and confusion matrix.

In [52]:
df1=df[df["class"]==1]
df4=df[df["class"]==4]
sub_df=pd.concat([df1,df4],axis=0)
df=df.drop(index=sub_df.index.tolist())
df.shape

(142, 19)

In [53]:
df.groupby("class")["class"].count()

class
2    81
3    61
Name: class, dtype: int64

In [54]:
y=df[["class"]]

In [55]:
X=df.iloc[:,2:19]
X.head()

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
1,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
2,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
4,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1


In [56]:

import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

# feature extraction. We are going to output the selection scores for all features and select the features with the highest scores.
test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
# summarize scores
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
16,49.096226
0,39.575078
8,27.807769
6,24.408524
13,22.666813
12,11.203638
14,8.251208
15,6.821163
5,4.233284
1,3.850400


In [57]:
X_scores=scores[scores["scores"]>=1]
len(X_scores)

12

In [11]:
#X=X.iloc[:,X_scores.index]
#X.head()

In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [13]:
BForest1=BalancedRandomForestClassifier(n_estimators=250,criterion="gini",max_depth=6,min_samples_split=3,
                                       min_samples_leaf=1,max_features="auto",bootstrap=True,
                                       sampling_strategy="majority",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest1,X,np.asarray(y).reshape(y.shape[0],),cv=12)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9002525252525252 +/- 0.060513924072332095


In [14]:
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(BForest1, X,np.asarray(y).reshape(y.shape[0],), cv=12)
AUC = auc(y, y_pred)
print("AUC: ",AUC)

AUC:  0.8994130742764624


In [15]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for the intial: \n",confusion_matrix(y,y_pred))

Confusion matrix for the intial: 
 [[74  7]
 [ 7 54]]


Let's find out what the model missed during the initial round of training and testing. 

In [18]:
y_pred=pd.DataFrame(y_pred).rename(columns={0:"Predicted label"})

In [19]:
y=y.reset_index(drop=True)

In [20]:
compare=pd.concat([y,y_pred],axis=1)
compare=pd.DataFrame(compare["class"]==compare["Predicted label"]).rename(columns={0:"Comparison"})
comparison=compare[compare["Comparison"]==False]
comparison

,Comparison
5,False
28,False
38,False
65,False
68,False
71,False
86,False
98,False
100,False
116,False


In [21]:
indices=comparison.index.tolist()

In [22]:
indices

[5, 28, 38, 65, 68, 71, 86, 98, 100, 116, 126, 130, 137, 141]

In [23]:
X=X.reset_index(drop=True)

In [24]:
missed=pd.concat([y,X],axis=1)
missed=missed.loc[indices,:]
missed

,class,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
5,2,1,1,1,1,1,1,2,1,3,3,3,3,6,3,1,2,4
28,3,1,1,1,1,1,1,1,1,2,2,4,2,8,2,2,2,1
38,3,1,1,1,1,2,1,2,1,2,2,3,3,3,3,1,2,1
65,3,1,1,1,1,1,1,1,1,2,3,2,3,3,2,2,1,1
68,3,1,1,1,1,1,1,1,1,2,2,2,2,5,1,1,2,2
71,3,1,1,1,1,2,1,1,1,2,2,2,2,3,1,1,1,1
86,3,2,2,1,2,2,1,2,1,1,2,2,1,3,1,2,2,2
98,2,1,1,1,1,1,1,2,1,2,2,2,2,4,1,2,1,2
100,2,1,1,1,1,1,1,2,1,3,3,2,2,4,2,2,2,2
116,2,1,1,1,1,1,1,1,1,1,2,4,3,8,2,2,2,2


In [25]:
missed.groupby("class")["class"].count()

class
2    7
3    7
Name: class, dtype: int64

In [26]:
missed.shape

(14, 18)

I am going to count the 2 instances of Class 1 as misclassified and include the 4 instances of Class 4 in the reinforcement round of training and testing.

In [28]:
df4=df4.drop(columns=1)
df4

,class,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
14,4,2,2,2,2,2,2,2,3,1,1,2,2,8,1,2,2,4
36,4,1,1,1,2,2,2,1,3,1,1,4,2,5,3,1,2,4
44,4,2,2,2,2,2,2,1,2,2,2,4,2,4,3,2,2,7
139,4,1,1,1,2,2,2,1,3,1,1,2,1,5,3,1,1,7


In [33]:
df2=pd.concat([df4,missed],axis=0)
df2.shape

(18, 18)

In [34]:
df2.groupby("class")["class"].count()

class
2    7
3    7
4    4
Name: class, dtype: int64

In [35]:
X2=df2.drop(["class"],axis=1)
y2=df2[["class"]]

In [36]:
BForest2=BalancedRandomForestClassifier(n_estimators=110,criterion="entropy",max_depth=5,min_samples_split=2,
                                       min_samples_leaf=2,max_features="auto",bootstrap=False,
                                       sampling_strategy="majority",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest2,X2,np.asarray(y2).reshape(y2.shape[0],),cv=4)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.8375 +/- 0.09601432184835759


In [37]:
from sklearn.metrics import confusion_matrix
y_pred2 = cross_val_predict(BForest2, X2,np.asarray(y2).reshape(y2.shape[0],), cv=4)
print("Confusion matrix for the reinforcement round: \n",confusion_matrix(y2,y_pred2))

Confusion matrix for the reinforcement round: 
 [[5 2 0]
 [1 6 0]
 [0 0 4]]


In [38]:
y2.groupby("class")["class"].count()

class
2    7
3    7
4    4
Name: class, dtype: int64

So, during the reinforcement round of training and testing, the model missed 3 instances. Including the 2 instances of Class 1 that the model could not handle and were therefore considered as misclassified, the model missed a total of 3+2=5 instances. **Hence, the model's accuracy is 96.62% [= ((148-5)/148)*100]**